# Model Evaluation
Will formalize into functions later

In [1]:
%load_ext autotime

In [2]:
import pandas as pd
import numpy as np
import pickle

import keras
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.models import load_model

from pathlib import Path
from matplotlib import pyplot as plt
%matplotlib inline

import utils

Using TensorFlow backend.


time: 4.26 s


In [3]:
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

time: 674 ms


In [22]:
x_test, y_test = utils.read_mura_pickle(sample='valid')

INFO:utils.utils:loading data/MURA-v1.1/x_valid.pkl
INFO:utils.utils:loading data/MURA-v1.1/y_valid.pkl


time: 302 ms


In [25]:
size = x_test.shape[1]
x_test = x_test.reshape(x_test.shape[0], size, size, 1)
x_test = utils.normalize_pixels(x_test)
y_test = to_categorical(y_test)

time: 437 ms


In [13]:
with tf.device('/GPU:1'):
    model = load_model('./trained_models/1_3_densenet201_mura_250.h5')

time: 42.1 s


In [14]:
y_test_hat = model.predict(x_test)

time: 27.4 s


In [26]:
true_label = np.argmax(y_test, axis=1)
pred_label = np.argmax(y_test_hat, axis=1)

time: 1.92 ms


In [62]:
from sklearn.metrics import confusion_matrix

class MURAMetrics():
    
    def __init__(self, true_label, pred_label):
        assert true_label.shape[0] == pred_label.shape[0], (
            'true_label and pred_laben must have the same length!')
        self.y = true_label
        self.yhat = pred_label
        self.N = true_label.shape[0]
        self.cm = confusion_matrix(true_label, pred_label)
        
    def accuracy(self):
        return (self.cm[0, 0] + self.cm[1, 1]) / self.N

    def kappa(self):
        p_observed = self.accuracy()
        p_expected = (
            self.cm[1, :].sum() * self.cm[:, 1].sum() 
            + self.cm[0, :].sum() * self.cm[:, 0].sum()) / self.N**2
        return (p_observed - p_expected) / (1 - p_expected)
    
    def precision_and_recall(self):
        return {
            'precision': self.cm[1, 1] / self.cm[:, 1].sum(),
            'recall': self.cm[1, 1] / self.cm[1, :].sum()
        }

time: 11.6 ms


In [63]:
x = ModelEval(true_label, pred_label)

time: 7.74 ms


In [64]:
x.accuracy()

0.7269314982796372

time: 4.12 ms


In [65]:
x.kappa()

0.4419623942875436

time: 4.18 ms


In [66]:
x.precision_and_recall()

{'precision': 0.9090909090909091, 'recall': 0.477124183006536}

time: 4.35 ms
